In [7]:
! pip install sentence-splitter
! pip install SentencePiece

In [25]:
!pip install sentencepiece

In [7]:
! pip install transformers

In [1]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [3]:
print("CUDA Available:", torch.cuda.is_available())

CUDA Available: True


In [5]:
model = PegasusForConditionalGeneration.from_pretrained('tuner007/pegasus_paraphrase')
tokenizer = PegasusTokenizer.from_pretrained('tuner007/pegasus_paraphrase')

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Tokenization

In [8]:
text = "The ultimate test of your knowledge is your capacity to convey it to another"

batch = tokenizer([text], padding = True, truncation = True, max_length = 60, return_tensors = 'pt')

output = model.generate(**batch, max_length = 60, num_beams = 5, num_return_sequences = 5, temperature = 1.5)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [10]:
results = tokenizer.batch_decode(output, skip_special_tokens = True)
results

['The test of your knowledge is your ability to convey it.',
 'Your capacity to convey your knowledge is the ultimate test of it.',
 'The ability to convey your knowledge is the ultimate test of your knowledge.',
 'The test of your knowledge is your ability to communicate it.',
 'Your capacity to convey your knowledge is the ultimate test.']

In [12]:
# save trained model and tokenizer
model.save_pretrained("Downloads")
tokenizer.save_pretrained("Downloads")

C:\Users\divya\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 60, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('Downloads\\tokenizer_config.json',
 'Downloads\\special_tokens_map.json',
 'Downloads\\spiece.model',
 'Downloads\\added_tokens.json')

In [14]:
# load saved model and tokenizer
model = PegasusForConditionalGeneration.from_pretrained('Downloads')
tokenizer = PegasusTokenizer.from_pretrained('Downloads')

# Predictive System (Generates ParaPhrases)

In [17]:
def get_response(input_text, num_return_sequences =5, num_beams = 5):
    batch = tokenizer([input_text], padding = True, truncation = True, max_length = 60, return_tensors = 'pt')
    Translated = model.generate(**batch, max_length = 60, num_beams = num_beams, num_return_sequences = num_return_sequences, temperature = 1.5)
    target_text = tokenizer.batch_decode(Translated, skip_special_tokens = True)

    return target_text

In [19]:
num_beams = 10
num_return_sequences = 8
context = "Machine learning is a growing flied in AI"
get_response(context, num_return_sequences, num_beams)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


['Artificial intelligence is growing in machine learning.',
 'Machine learning is growing.',
 'Machine learning is gaining in popularity.',
 'Machine learning is getting better.',
 'Machine learning is a growing field.',
 'Machine learning is becoming more and more popular.',
 'Machine learning is gaining popularity.',
 'Machine learning is growing in popularity.']

In [21]:
# Taking inputs from user:
print("\n PEGASUS Paraphraser \n")

user_input = input("🔹 Enter a sentence to generate paraphrase:\n> ")

num_outputs = int(input("\n🔹 How many paraphrases do you want? (e.g., 3 to 10):\n> "))
beam_width = int(input("🔹 Enter beam search width (Should be equal or greater to the number of paraphrases you want to generate):\n> "))

# Generating and showing results here:
print("\n Generating Paraphrases...\n")
paraphrased_sentences = get_response(user_input, num_outputs, beam_width)

for i, sent in enumerate(paraphrased_sentences, 1):
    print(f"{i}. {sent}")



 PEGASUS Paraphraser 



🔹 Enter a sentence to generate paraphrase:
>  Lion is a powerful predator and is known as the "King of the Jungle" due to its strength and dominance.

🔹 How many paraphrases do you want? (e.g., 3 to 10):
>  10
🔹 Enter beam search width (Should be equal or greater to the number of paraphrases you want to generate):
>  10


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



 Generating Paraphrases...

1. Lion is known as the "King of the Jungle" due to its strength and dominance.
2. The lion is known as the "King of the Jungle" due to its strength and dominance.
3. The Lion is known as the "King of the Jungle" due to its strength and dominance.
4. Lion is known as the "King of the Jungle" because of its strength and dominance.
5. The lion is known as the "King of the Jungle" because of its strength and dominance.
6. Lion is the "King of the Jungle" due to its strength and dominance.
7. The Lion is known as the "King of the Jungle" because of its strength and dominance.
8. The lion is known as the "King of the Jungle" due to its strength.
9. Lion is known as the "King of the Jungle" due to its strength.
10. Lion is the "King of the Jungle" because of its strength and dominance.
